#installing dependencies

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB
Sat May 29 12:00:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                

In [ ]:
!pip install optuna==2.3.0
!pip install transformers==4.2.1
!pip install farasapy
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert

     |████████████████████████████████| 266kB 7.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 174kB 13.0MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 112kB 12.8MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 143kB 12.7MB/s 
  Created wheel for optuna: filename=optuna-2.3.0-cp37-none-any.whl size=359761 sha256=d665c4d0c466794dc752e6de893806597916a33dd76171fb856bd6fd9d0bf20b
  Stored in directory: /root/.cache/pip/wheels/fa/91/19/64b0ec6b964f89c0695a9dc6db6f851d0b54c5381a5c9cadfb
Successfully built optuna
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=627ca318be90b19bb94d16089db956546c9b7204de9403b5ee4f80aa109b96eb
  Stored in directory: /root/.cache/pip/wheels/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Creating training datasets

In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
all_datasets= []

In [ ]:
class Dataset:
    def __init__(
        self,
        name,
        train,
        test,
        label_list,
    ):
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list

In [ ]:
DATA_COLUMN = "cleaned_text"
LABEL_COLUMN = "Class_camel"

##HARD - Balanced

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Omdena_sentiment/Dataset/train.csv')

df = df[[DATA_COLUMN, LABEL_COLUMN]]  # we are interested in rating and review only
# df_HARD.columns = [DATA_COLUMN, LABEL_COLUMN]
print(df[LABEL_COLUMN].value_counts())


positive    57096
negative    33702
neutral     20124
Name: Class_camel, dtype: int64


In [ ]:
val_df=pd.read_csv('/content/drive/MyDrive/Omdena_sentiment/Dataset/val.csv')
val_df = val_df[[DATA_COLUMN, LABEL_COLUMN]]

In [ ]:
# code rating as +ve if > 3, -ve if less, no 3s in dataset
categories = ['neutral', 'negative', 'positive'] #classes present in the data
data_Hard = Dataset("HARD", df, val_df, categories)
all_datasets.append(data_Hard)

#Trainer

In [ ]:
from arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample
import logging
import torch
import optuna 

In [ ]:
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

In [ ]:
for x in all_datasets:
  print(x.name)

HARD


You can choose which model, and dataset from here along with the max sentence length

In [ ]:
dataset_name = 'HARD'
model_name = 'aubmindlab/bert-large-arabertv02'
task_name = 'classification'
max_len = 128

In [ ]:
for d in all_datasets:
  if d.name==dataset_name:
    selected_dataset = d
    print('Dataset found')
    break

Dataset found


In [ ]:
arabert_prep = ArabertPreprocessor(model_name.split("/")[-1])

selected_dataset.train[DATA_COLUMN] = selected_dataset.train[DATA_COLUMN].apply(arabert_prep.preprocess)
selected_dataset.test[DATA_COLUMN] = selected_dataset.test[DATA_COLUMN].apply(arabert_prep.preprocess)  

In [ ]:
selected_dataset.test[DATA_COLUMN]

0                                   حقوق المراه في الاسلام
1                       اومن حقوق المراه انك تفتحلها الباب
2        نعم المراه في الاسلام منتقصه الحقوق والوااجبات...
3        محاولات بائسه لاقصاء دور الام السعوديه ودورها ...
4                                         ناصر حقوق المراه
                               ...                        
27793    رحم الله شهداء الواجب الوطني علي حدودنا وادعو ...
27794                     باقي ايام اللهم بلغنا رمضان امين
27795                                   الله حي التاني جاي
27796    عبد المنعم ابو الفتوح في علمتني الحياه الجزء ا...
27797    عبدالمنعم عطيه عضو مجلس اداره نادي دمنهوروزراء...
Name: cleaned_text, Length: 27798, dtype: object

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

In [ ]:
label_map = { v:index for index, v in enumerate(selected_dataset.label_list) }
print(label_map)
train_dataset = BERTDataset(selected_dataset.train[DATA_COLUMN].to_list(),selected_dataset.train[LABEL_COLUMN].to_list(),model_name,max_len,label_map)
test_dataset = BERTDataset(selected_dataset.test[DATA_COLUMN].to_list(),selected_dataset.test[LABEL_COLUMN].to_list(),model_name,max_len,label_map)

{'neutral': 0, 'negative': 1, 'positive': 2}


In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [ ]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  #print(classification_report(p.label_ids,preds))
  #print(confusion_matrix(p.label_ids,preds))

  macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[0,1])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {
      'macro_f1' : macro_f1,
      'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
  }

#Regular Training

This paert allows you to do a regular training with no hyper parameter optimization

In [ ]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 5e-5
training_args.fp16 = True
training_args.per_device_train_batch_size = 16
training_args.per_device_eval_batch_size = 16
training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 8


steps_per_epoch = (len(selected_dataset.train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)
#Warmup_ratio
warmup_ratio = 0.1
training_args.warmup_steps = total_steps*warmup_ratio # or you can set the warmup steps directly 

training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000 #don't want to save any model, there is probably a better way to do this :)
training_args.seed = 42
training_args.disable_tqdm = False
training_args.lr_scheduler_type = 'cosine'

3466
27728


In [ ]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Macro F1,Macro F1 Pos Neg,Macro Precision,Macro Recall,Accuracy,Runtime,Samples Per Second
0,0.457000,0.421886,0.794500,0.749709,0.811993,0.782996,0.831499,253.973300,109.452000
1,0.347100,0.419059,0.811983,0.769516,0.826872,0.806472,0.845888,253.735000,109.555000
2,0.216800,0.464089,0.820282,0.781177,0.823453,0.817440,0.849522,254.132800,109.384000
3,0.127000,0.558509,0.821143,0.781500,0.820236,0.823832,0.849054,254.142300,109.380000
4,0.080200,0.672345,0.822831,0.783450,0.836231,0.812335,0.854054,254.112600,109.392000
5,0.042600,0.872644,0.823913,0.783926,0.833597,0.816260,0.855529,254.133700,109.383000
6,0.024900,0.957822,0.825621,0.786594,0.834959,0.818218,0.856680,254.135000,109.383000
7,0.010100,1.004895,0.825720,0.786448,0.832914,0.819526,0.856213,254.130100,109.385000


TrainOutput(global_step=27728, training_loss=0.17174688206455713, metrics={'train_runtime': 25518.5519, 'train_samples_per_second': 1.087, 'total_flos': 184270884933528576, 'epoch': 8.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/Omdena_sentiment/Saved_models/AraBERT/")

In [ ]:
import os

In [ ]:
if not os.path.exists('/content/drive/MyDrive/Omdena_sentiment/Saved_models/AraBERT/'):
  os.mkdir('/content/drive/MyDrive/Omdena_sentiment/Saved_models/AraBERT/')

In [ ]:
test_df=pd.read_csv('/content/drive/MyDrive/Omdena_sentiment/Dataset/test.csv')
test_df = val_df[[DATA_COLUMN, LABEL_COLUMN]]

In [ ]:
test_df[DATA_COLUMN] = test_df[DATA_COLUMN].apply(arabert_prep.preprocess)



In [ ]:
test_dataset = BERTDataset(test_df[DATA_COLUMN].to_list(),test_df[LABEL_COLUMN].to_list(),model_name,max_len,label_map)


In [ ]:
trainer.evaluate(test_dataset)

{'epoch': 8.0,
 'eval_accuracy': 0.8562126771710195,
 'eval_loss': 1.004894733428955,
 'eval_macro_f1': 0.8257200735663509,
 'eval_macro_f1_pos_neg': 0.7864475386082397,
 'eval_macro_precision': 0.8329137531542009,
 'eval_macro_recall': 0.8195258109194047,
 'eval_runtime': 254.1375,
 'eval_samples_per_second': 109.382}